# Training TF net

In [1]:
import sys
sys.path.insert(0, '/braintree/home/bashivan/dropbox/Codes/3D_fMRI_CNN/tf_pipeline/')
import os
from tf_trainer import Trainer
import logging 
from tf_model import TFModel
from tf_dataset import TFDataset
import numpy as np
import tensorflow as tf
import numpy as np
FLAGS = tf.app.flags.FLAGS
os.environ['CUDA_VISIBLE_DEVICES'] = '0,2'

FLAGS.initial_learning_rate = 0.001
FLAGS.batch_size = 64
FLAGS.num_gpus = 2
FLAGS.num_epochs_per_decay = 3
FLAGS.num_time_steps = 64

fold_to_run = 0

# Load the dataset
fold_pairs = []

model = TFModel()
dataset = TFDataset(data_dir='/braintree/data2/active/users/bashivan/Data/fmri_conv_orig/')
tr = Trainer(model=model, dataset=dataset)
print("Loading data...")
tr.load_data(random=False)

sub_nums = tr.subjects
subs_in_fold = np.ceil(np.max(sub_nums) / float(10))
# n-fold cross validation
for i in range(FLAGS.num_folds):
    '''
    for each kfold selects fold window to collect indices for test dataset and the rest becomes train
    '''
    test_ids = np.bitwise_and(sub_nums >= subs_in_fold * (i), sub_nums < subs_in_fold * (i + 1))
    train_ids = ~ test_ids
    fold_pairs.append((np.nonzero(train_ids)[0], np.nonzero(test_ids)[0]))

    

Loading data...


In [2]:
FLAGS = tf.app.flags.FLAGS

fold_num = 0
fold = fold_pairs[0]

FLAGS.train_dir = os.path.join(FLAGS.train_dir, str(fold_num))

print('Splitting the data...')
tr.split_data(fold)
print('Preprocessing data...')
tr.preprocess_data()


Splitting the data...
Preprocessing data...


In [3]:
tr.train(fold_num=fold_num)


****************************************
subset: train
data_dir: /braintree/data2/active/users/bashivan/Data/fmri_conv_orig
num_epochs_per_decay: 3
num_gpus: 2
train_dir: /braintree/data2/active/users/bashivan/results/temp/0
batch_size: 64
model_type: lstm
num_checkpoints_tosave: 5
initial_learning_rate: 0.001
num_examples: None
num_epochs: 10
checkpoint_dir: .
learning_rate_decay_factor: 0.1
fold_to_run: -1
num_folds: 10
seed: 0
eval_dir: /om/user/bashivan/temp
num_time_steps: 64
log_device_placement: False
****************************************
> /braintree/home/bashivan/dropbox/Codes/3D_fMRI_CNN/tf_pipeline/tf_dataset.py(18)num_examples_per_epoch()
-> if subset == 'train':
(Pdb) (self._features.shape[0] / FLAGS.num_folds * (FLAGS.num_folds - 2)) * \              (self._features.shape[-1] - FLAGS.num_time_steps)
*** SyntaxError: unexpected character after line continuation character (<stdin>, line 1)
(Pdb) (self._features.shape[0] / FLAGS.num_folds * (FLAGS.num_folds - 2)) * (self.

0

In [1]:
import cPickle
data = cPickle.load(open('/braintree/home/bashivan/dropbox/Codes/3D_fMRI_CNN/cnn_lstm_results_adam_0.0001_10fold_.pkl'))

data.keys()

Index([u'test_acc', u'test_loss', u'training_acc', u'training_loss',
       u'valid_acc', u'valid_loss'],
      dtype='object')

In [2]:
data['test_acc']

0    0.553496
1    0.607521
2    0.607521
3    0.627913
4    0.627913
Name: test_acc, dtype: float64